In [1]:
1+1

2

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()
import os

api_key=os.getenv("GAMINI_API_KEY")


In [3]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key,
)

In [4]:
dsa_solver=AssistantAgent(
    name="dsa_solver",
    model_client=model_client,
    system_message="you give me code in c++ to solve complex dsa problem"
)

code_reviewer=AssistantAgent(
    name="recviewer",
    model_client=model_client,
    system_message="you review the code by the dsa_solver "
)
code_editer=AssistantAgent(
    name="code_editer",
    model_client=model_client,
    system_message="you make the code easy to understand and add comment of the code "
)

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat
team=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editer],
    max_turns=3
)

In [6]:
from autogen_agentchat.messages import TextMessage

async def test_team():
    task=TextMessage(content="write code to of find prime number in c++",source='user')
    result= await team.run(task=task)

    for each_messages in result.messages:
        print(f'{each_messages.source}:{each_messages.content}')

In [8]:
await test_team()

user:write code to of find prime number in c++
dsa_solver:```C++
#include <iostream>
#include <vector>
#include <cmath>
#include <limits>  // For numeric_limits

using namespace std;

bool isPrime(int n) {
  // Handle base cases efficiently.
  if (n <= 1) return false;
  if (n <= 3) return true;
  if (n % 2 == 0 || n % 3 == 0) return false;

  // Optimized loop: Check divisibility by 6k ± 1.
  for (int i = 5; i * i <= n; i = i + 6) {
    if (n % i == 0 || n % (i + 2) == 0) {
      return false;
    }
  }
  return true;
}

// Function to find prime numbers up to a limit (Sieve of Eratosthenes)
vector<int> sieveOfEratosthenes(int limit) {
    if (limit < 2) return {}; // Empty vector for invalid input.

    vector<bool> isPrime(limit + 1, true);
    isPrime[0] = isPrime[1] = false;

    for (int p = 2; p * p <= limit; ++p) {
        if (isPrime[p]) {
            for (int i = p * p; i <= limit; i += p) {
                isPrime[i] = false;
            }
        }
    }

    vector<int> pr

In [10]:
from autogen_agentchat.base import TaskResult
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="how to find even or odd code"):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.content)

how to find even or odd code
There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:

**Method 1: Using the modulo operator (%)**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if (number % 2 == 0) {
    cout << number << " is even." << endl;
  } else {
    cout << number << " is odd." << endl;
  }

  return 0;
}
```

* **Explanation:** The modulo operator (%) gives you the remainder of a division. If a number is perfectly divisible by 2 (remainder is 0), it's even. Otherwise, it's odd.

* **Advantages:** This is the most straightforward and common approach.  It's efficient and easy to understand.

* **Disadvantages:**  None significant in this simple context.


**Method 2: Using a bitwise AND operator (&)**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if ((number & 1)

In [12]:
from autogen_agentchat.ui import Console
await team.reset()
await Console(team.run_stream(task="how to find even or odd code"))

---------- TextMessage (user) ----------
how to find even or odd code


---------- TextMessage (dsa_solver) ----------
There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:

**1. Using the modulo operator (%)**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if (number % 2 == 0) {
    cout << number << " is even." << endl;
  } else {
    cout << number << " is odd." << endl;
  }

  return 0;
}
```

* **Explanation:** The modulo operator (%) returns the remainder of a division.  If a number is perfectly divisible by 2 (remainder 0), it's even; otherwise, it's odd.

* **Pros:** Concise and efficient.  This is the most common and generally preferred approach.

* **Cons:**  No significant drawbacks.


**2. Using a bitwise AND operation**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if ((number & 1) == 0) {
    cout << number << " is even." <<

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 17, 49, 51, 968999, tzinfo=datetime.timezone.utc), content='how to find even or odd code', type='TextMessage'), TextMessage(source='dsa_solver', models_usage=RequestUsage(prompt_tokens=20, completion_tokens=607), metadata={}, created_at=datetime.datetime(2025, 6, 22, 17, 49, 55, 690664, tzinfo=datetime.timezone.utc), content='There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:\n\n**1. Using the modulo operator (%)**\n\n```C++\n#include <iostream>\n\nusing namespace std;\n\nint main() {\n  int number;\n\n  cout << "Enter an integer: ";\n  cin >> number;\n\n  if (number % 2 == 0) {\n    cout << number << " is even." << endl;\n  } else {\n    cout << number << " is odd." << endl;\n  }\n\n  return 0;\n}\n```\n\n* **Explanation:** The modulo operator (%) returns the remainder of a division.  If a number is perfectly 